In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import csv
import cv2
import datetime
import ipynbname
import math
import numpy
import os
import pandas
import pathlib
import skimage.io
import skimage.transform
import sklearn.model_selection
import tensorflow
import time

from albumentations import (
    Compose, HorizontalFlip, ShiftScaleRotate, ElasticTransform,
    RandomBrightness, RandomContrast, RandomGamma
)

from files import create_folder, save_fit_history, save_lossgraph, save_figs
from metrics import dice_coef, jaccard_distance
from model import evaluate, unet_model, get_loss_function
from AugmentationSequence import AugmentationSequence

In [ ]:
class CreateSequence(tensorflow.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(numpy.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return batch_x, batch_y

# GPU

In [ ]:
gpus = tensorflow.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            print(f"GPU: {gpu.name}")
            tensorflow.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
cfg = {
    "channel": 3,
    "batch_size": 4,
    "fold": 5,
    "epochs": 75,
    "image_size": 256,
    "learning_rate": 0.001,
    "random_state": 1234,
    "test_size": 0.2,
    "val_size": 0.05,
    "path_dataset": "dataset",
    "path_out": "out",
    "loss_function": "dice",
    "data_augmentation": False
}
images_folder = os.path.join(cfg["path_dataset"], "IMAGEM_ORIGINAL", "CONVERTIDAS", "RGB", "256", "OUT")
masks_folder = os.path.join(cfg["path_dataset"], "MASK", "BITMAP", "256", "OUT")

In [ ]:
list_labels = list([])
list_images = list([])
list_images_names = list([])
def load_files():
    for file in sorted(pathlib.Path(masks_folder).rglob("*")):
        mask = skimage.io.imread(file.resolve())
        mask = numpy.float32(mask/255)
        list_labels.append(mask)

        image = skimage.io.imread(os.path.join(images_folder, f"{file.stem}.png"))
        image = numpy.float32(image/255)
        list_images.append(image)

        list_images_names.append(str(file.stem))

load_files()
print(len(list_labels), len(list_images), len(list_images_names))

In [ ]:
x = numpy.array(list_images).reshape((len(list_images), cfg["image_size"], cfg["image_size"], cfg["channel"]))
y = numpy.array(list_labels).reshape((len(list_labels), cfg["image_size"], cfg["image_size"], 1))

print(x.shape, y.shape)

In [ ]:
kf = sklearn.model_selection.KFold(n_splits=cfg["fold"], shuffle=True, random_state=cfg["random_state"])

models = []
list_evaluate = list([])
current_datetime = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
path = os.path.join(cfg["path_out"], current_datetime)
create_folder(list([path]))
for fold, (train_index, test_index) in enumerate(kf.split(x)):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(x_train, y_train, test_size=cfg["val_size"], random_state=cfg["random_state"])

    print(x_train.shape)
    print(x_val.shape)
    print(x_test.shape)
    print(x.shape)

    path_fold = os.path.join(path, str(fold))
    create_folder(list([path_fold]))

    augment = Compose([
        HorizontalFlip(),
        ShiftScaleRotate(rotate_limit=45, border_mode=cv2.BORDER_CONSTANT),
        ElasticTransform(border_mode=cv2.BORDER_CONSTANT),
        RandomBrightness(),
        RandomContrast(),
        RandomGamma()
    ])
    steps_per_epoch = math.ceil(x_train.shape[0] / cfg["batch_size"])
    if cfg["data_augmentation"]:
        train_generator = AugmentationSequence(x_train, y_train, cfg["batch_size"], augment)
    else:
        train_generator = CreateSequence(x_train, y_train, cfg["batch_size"])
    reduce_learning_rate = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, verbose=1)
    filename_model = os.path.join(path_fold, "unet.h5")
    checkpointer = tensorflow.keras.callbacks.ModelCheckpoint(filename_model, verbose=1, save_best_only=True)
    strategy = tensorflow.distribute.MirroredStrategy()

    with strategy.scope():
        model = unet_model(cfg)
        adam_opt = tensorflow.keras.optimizers.Adam(learning_rate=cfg["learning_rate"])
        model.compile(optimizer=adam_opt, loss=get_loss_function(cfg["loss_function"]), metrics=[dice_coef, jaccard_distance, tensorflow.keras.metrics.Precision(), tensorflow.keras.metrics.Recall()])

    tensorflow.keras.backend.clear_session()
    start_time = time.time()
    fit = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=cfg["epochs"], validation_data=(x_val, y_val), callbacks=[checkpointer, reduce_learning_rate])
    end_time = time.time() - start_time

    save_fit_history(fold, fit, path_fold)
    save_lossgraph(fold, fit, path_fold)
    list_evaluate.append(evaluate(end_time, fold, model, x_train, x_val, x_test, y_train, y_val, y_test))

    models.append(model)

    # model = tensorflow.keras.models.load_model("unet_rgb.h5", custom_objects = {"dice_loss": dice_loss, "dice_coef": dice_coef, "jaccard_distance": jaccard_distance })

    save_figs(cfg, list_images_names, test_index, model, path_fold, x)

tensorflow.keras.backend.clear_session()

In [ ]:
columns = ["batch_size", "epochs", "learning_rate", "loss_function", "images", "masks", "len_images", "len_masks", "channel", "image_size", "fold", "test_size", "val_size", "random_state", "path_dataset", "path_out", "data_augmentation", "filename_script"]
data = [cfg["batch_size"], cfg["epochs"], cfg["learning_rate"], cfg["loss_function"], images_folder, masks_folder, len(list_images), len(list_labels), cfg["channel"], cfg["image_size"], cfg["fold"], cfg["test_size"], cfg["val_size"], cfg["random_state"], cfg["path_dataset"], cfg["path_out"], cfg["data_augmentation"], os.path.realpath(__file__)]

dataframe_cfg = pandas.DataFrame(data, columns)
dataframe_cfg = dataframe_cfg.applymap(lambda x: str(x).replace(".", ",") if isinstance(x,float) else x)
dataframe_cfg.to_csv(os.path.join(path, "cfg.csv"), decimal=",", sep=";", na_rep=" ", header=False, quoting=csv.QUOTE_ALL)
dataframe_cfg

In [ ]:
def get_mean(key, list_evaluate):
    return str(numpy.mean(list([evaluate[key] for evaluate in list_evaluate])))

def get_std(key, list_evaluate):
    return str(numpy.std(list([evaluate[key] for evaluate in list_evaluate])))

dataframe_mean = pandas.DataFrame({"mean_train": [get_mean("loss_train", list_evaluate),
                                                  get_mean("dice_train", list_evaluate),
                                                  get_mean("jaccard_train", list_evaluate),
                                                  get_mean("precision_train", list_evaluate),
                                                  get_mean("recall_train", list_evaluate)],
                                   "std_train": [get_std("loss_train", list_evaluate),
                                                  get_std("dice_train", list_evaluate),
                                                  get_std("jaccard_train", list_evaluate),
                                                  get_std("precision_train", list_evaluate),
                                                  get_std("recall_train", list_evaluate)],
                                   "mean_val": [get_mean("loss_val", list_evaluate),
                                                  get_mean("dice_val", list_evaluate),
                                                  get_mean("jaccard_val", list_evaluate),
                                                  get_mean("precision_val", list_evaluate),
                                                  get_mean("recall_val", list_evaluate)],
                                   "std_val": [get_std("loss_val", list_evaluate),
                                                  get_std("dice_val", list_evaluate),
                                                  get_std("jaccard_val", list_evaluate),
                                                  get_std("precision_val", list_evaluate),
                                                  get_std("recall_val", list_evaluate)],
                                   "mean_test": [get_mean("loss_test", list_evaluate),
                                                  get_mean("dice_test", list_evaluate),
                                                  get_mean("jaccard_test", list_evaluate),
                                                  get_mean("precision_test", list_evaluate),
                                                  get_mean("recall_test", list_evaluate)],
                                   "std_test": [get_std("loss_test", list_evaluate),
                                                  get_std("dice_test", list_evaluate),
                                                  get_std("jaccard_test", list_evaluate),
                                                  get_std("precision_test", list_evaluate),
                                                  get_std("recall_test", list_evaluate)],
                                   }, index=["loss", "dice", "jaccard", "precision", "recall"])


dataframe_mean = dataframe_mean.applymap(lambda x: str(x).replace(".", ",") if isinstance(x,float) else x)
dataframe_mean = dataframe_mean.transpose()
dataframe_mean.to_csv(os.path.join(path, "mean.csv"), decimal=",", sep=";", na_rep=" ", quoting=csv.QUOTE_ALL)
dataframe_mean

In [ ]:
for evaluate in list_evaluate:
    filename_fold = os.path.join(path, str(evaluate["fold"]), "metrics.csv")

    dataframe_fold = pandas.DataFrame({"metrics_train": [evaluate["loss_train"],
                                                  evaluate["dice_train"],
                                                  evaluate["jaccard_train"],
                                                  evaluate["precision_train"],
                                                  evaluate["recall_train"]],
                                   "metrics_val": [evaluate["loss_val"],
                                                  evaluate["dice_val"],
                                                  evaluate["jaccard_val"],
                                                  evaluate["precision_val"],
                                                  evaluate["recall_val"]],
                                   "metrics_test": [evaluate["loss_test"],
                                                  evaluate["dice_test"],
                                                  evaluate["jaccard_test"],
                                                  evaluate["precision_test"],
                                                  evaluate["recall_test"]],
                                   }, index=["loss", "dice", "jaccard", "precision", "recall"])

    dataframe_fold_info = pandas.DataFrame({evaluate["fold"], evaluate["time"]}, index=["fold", "time"], columns=["info"])
    dataframe_fold = dataframe_fold.applymap(lambda x: str(x).replace(".", ",") if isinstance(x,float) else x)
    dataframe_fold = pandas.concat([dataframe_fold, dataframe_fold_info], axis=0)
    dataframe_fold = dataframe_fold.transpose()
    dataframe_fold.to_csv(filename_fold, decimal=",", sep=";", na_rep=" ", quoting=csv.QUOTE_ALL)

dataframe_fold